In [1]:
# 경고(warning) 비표시 
import warnings
warnings.filterwarnings('ignore')

In [6]:
# 교재 3장, (교재 p6,68 제외 by code error)
## 제가 운영하는 스포츠센터는 트레이닝 붐 덕분에 지금까지 고객 수가 늘었습니다. 
## 그런데 최근 1년간 고객 수가 늘지 않는 것 같습니다. 
## 자주 이용하는 고객은 계속 오지만, 가끔 오는 고객은 어느새 오지 않는 경우도 생기는 것 같습니다. 
## 제대로 데이터를 분석한 적이 없어서 어떤 고객이 계속 이용하고 있는지조차 모릅니다. 
## 데이터분석을 하면 뭔가 알 수 있을까요?
## 2018.4 ~ 2019.3 까지의 1년 Data

## 이용 이력 데이터 집계
## 정기 이용 플래그 작성
## 고객 데이터와 이용 이력 데이터 결합
## 회원 기간 계산
## 각종 통계량 파악
## 탈퇴 회원과 지속 회원의 차이를 파악

# use_log.csv : 이용이력, 기간은 2018.04~2019.03
# customer_master2.csv : 2019년 3월 말 시점의 회원 데이터 (이전 탈퇴 회원 포함)
# class_master.csv : 회원 구분 데이터(종일, 주간, 야간)
# campaign_master.csv : 행사 구분 데이터(입회비 유무 등)

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='malgun gothic')
plt.rcParams['axes.unicode_minus'] = False

In [13]:
uselog = pd.read_csv('Data/use_log_1.csv')
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [14]:
customer = pd.read_csv('Data/customer_master2_1.csv')
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [15]:
class_master = pd.read_csv('Data/class_master_1.csv')
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


In [16]:
campaign_master = pd.read_csv('Data/campaign_master_1.csv')
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


### 고겍 Data 가공

In [17]:
customer_join1 =  pd.merge(customer, class_master, on = 'class', how = 'left')
customer_join1.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000


In [18]:
print(len(customer_join1))

4192


In [19]:
customer_join2 =  pd.merge(customer_join1, campaign_master, on = 'campaign_id', how = 'left')
customer_join2.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반


In [20]:
print(len(customer_join2))

4192


In [21]:
customer_join2.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

In [23]:
customer_join2.groupby('class_name').count()['customer_id']

class_name
0_종일    2045
1_주간    1019
2_야간    1128
Name: customer_id, dtype: int64

In [24]:
customer_join2.groupby('campaign_id').count()['customer_id']

campaign_id
CA1    3050
CA2     650
CA3     492
Name: customer_id, dtype: int64

In [25]:
customer_join2.groupby('gender').count()['customer_id']

gender
F    1983
M    2209
Name: customer_id, dtype: int64

In [26]:
customer_join2.groupby('is_deleted').count()['customer_id']

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [27]:
customer_join2.groupby('class_name').count()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,price,campaign_name
class_name,,,,,,,,,,
0_종일,2045,2045,2045,2045,2045,664,2045,2045,2045,2045
1_주간,1019,1019,1019,1019,1019,344,1019,1019,1019,1019
2_야간,1128,1128,1128,1128,1128,342,1128,1128,1128,1128


In [28]:
customer_join2.groupby('class_name').sum()[['is_deleted','price']]

,is_deleted,price
class_name,,
0_종일,664,21472500
1_주간,344,7642500
2_야간,342,6768000


In [29]:
customer_join2.groupby('campaign_name').count()['customer_id']

campaign_name
0_입회비반액할인     650
1_입회비무료       492
2_일반         3050
Name: customer_id, dtype: int64

### 최근 Data 집계

In [30]:
customer_join2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    4192 non-null   object
 1   name           4192 non-null   object
 2   class          4192 non-null   object
 3   gender         4192 non-null   object
 4   start_date     4192 non-null   object
 5   end_date       1350 non-null   object
 6   campaign_id    4192 non-null   object
 7   is_deleted     4192 non-null   int64 
 8   class_name     4192 non-null   object
 9   price          4192 non-null   int64 
 10  campaign_name  4192 non-null   object
dtypes: int64(2), object(9)
memory usage: 360.4+ KB


In [32]:
###현재이용중인 회원

customer_join2['end_date'] = pd.to_datetime(customer_join2['end_date'])

In [34]:
customer_new = customer_join2.loc[(customer_join2['end_date']>= pd.to_datetime('20190331')) | (customer_join2['end_date'].isna())]

In [35]:
print(len(customer_new))

2953


In [36]:
customer_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2953 entries, 0 to 4191
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    2953 non-null   object        
 1   name           2953 non-null   object        
 2   class          2953 non-null   object        
 3   gender         2953 non-null   object        
 4   start_date     2953 non-null   object        
 5   end_date       111 non-null    datetime64[ns]
 6   campaign_id    2953 non-null   object        
 7   is_deleted     2953 non-null   int64         
 8   class_name     2953 non-null   object        
 9   price          2953 non-null   int64         
 10  campaign_name  2953 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 276.8+ KB


In [37]:
customer_new['end_date'].unique()

<DatetimeArray>
['NaT', '2019-03-31 00:00:00']
Length: 2, dtype: datetime64[ns]

In [38]:
customer_new.groupby('class_name').count()['customer_id']

class_name
0_종일    1444
1_주간     696
2_야간     813
Name: customer_id, dtype: int64

In [39]:
customer_new.groupby('campaign_name').count()['customer_id']

campaign_name
0_입회비반액할인     311
1_입회비무료       242
2_일반         2400
Name: customer_id, dtype: int64

In [40]:
customer_new.groupby('gender').count()['customer_id']

gender
F    1400
M    1553
Name: customer_id, dtype: int64

In [41]:
customer_new.groupby('is_deleted').count()['customer_id']

is_deleted
0    2842
1     111
Name: customer_id, dtype: int64

###이용이력 Data 집계

In [42]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   log_id       197428 non-null  object
 1   customer_id  197428 non-null  object
 2   usedate      197428 non-null  object
dtypes: object(3)
memory usage: 4.5+ MB


In [43]:
uselog['usedate'] = pd.to_datetime(uselog['usedate'])

In [44]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   log_id       197428 non-null  object        
 1   customer_id  197428 non-null  object        
 2   usedate      197428 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 4.5+ MB


In [45]:
uselog['년월']  = uselog['usedate'].dt.strftime('%Y%m')

In [46]:
uselog.head()

,log_id,customer_id,usedate,년월
0,L00000049012330,AS009373,2018-04-01,201804
1,L00000049012331,AS015315,2018-04-01,201804
2,L00000049012332,AS040841,2018-04-01,201804
3,L00000049012333,AS046594,2018-04-01,201804
4,L00000049012334,AS073285,2018-04-01,201804


In [48]:
uselog.groupby('년월').count()[['customer_id','log_id']]

,customer_id,log_id
년월,,
201804,16460,16460
201805,16913,16913
201806,16994,16994
201807,16954,16954
201808,16674,16674
201809,16471,16471
201810,15989,15989
201811,15697,15697
201812,16383,16383


In [50]:
uselog_months = uselog.groupby(['년월', 'customer_id']).count()
uselog_months

log_id  usedate
년월     customer_id                 
201804 AS002855          4        4
       AS009013          2        2
       AS009373          3        3
       AS015315          6        6
       AS015739          7        7
...                    ...      ...
201903 TS995853          8        8
       TS998593          8        8
       TS999079          3        3
       TS999231          6        6
       TS999855          4        4

[36842 rows x 2 columns]

In [52]:
uselog_months = uselog.groupby(['년월', 'customer_id'], as_index=False).count()
uselog_months

,년월,customer_id,log_id,usedate
0,201804,AS002855,4,4
1,201804,AS009013,2,2
2,201804,AS009373,3,3
3,201804,AS015315,6,6
4,201804,AS015739,7,7
...,...,...,...,...
36837,201903,TS995853,8,8
36838,201903,TS998593,8,8
36839,201903,TS999079,3,3
36840,201903,TS999231,6,6


In [53]:
uselog_months.rename(columns={'log_id':'count'}, inplace = True ) #열이름변경
del uselog_months['usedate'] #열 삭제
uselog_months

,년월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [56]:
uselog_customer = uselog_months.groupby('customer_id').agg(['mean','median','max','min'])['count']

In [58]:
uselog_customer

,mean,median,max,min
customer_id,,,,
AS002855,4.500000,5.0,7,2
AS008805,4.000000,4.0,8,1
AS009013,2.000000,2.0,2,2
AS009373,5.083333,5.0,7,3
AS015233,7.545455,7.0,11,4
...,...,...,...,...
TS995853,9.500000,9.5,11,8
TS998593,8.142857,8.0,9,7
TS999079,4.916667,5.5,9,2


In [59]:
uselog_customer = uselog_customer.reset_index(drop=False)
uselog_customer

,customer_id,mean,median,max,min
0,AS002855,4.500000,5.0,7,2
1,AS008805,4.000000,4.0,8,1
2,AS009013,2.000000,2.0,2,2
3,AS009373,5.083333,5.0,7,3
4,AS015233,7.545455,7.0,11,4
...,...,...,...,...,...
4187,TS995853,9.500000,9.5,11,8
4188,TS998593,8.142857,8.0,9,7
4189,TS999079,4.916667,5.5,9,2
4190,TS999231,4.666667,5.0,8,1


In [60]:
## 이용이력 Data로 정기이용 여부 플래그 작성

uselog['weekday'] = uselog['usedate'].dt.weekday
uselog # 월 ~ 일 0~ 6

,log_id,customer_id,usedate,년월,weekday
0,L00000049012330,AS009373,2018-04-01,201804,6
1,L00000049012331,AS015315,2018-04-01,201804,6
2,L00000049012332,AS040841,2018-04-01,201804,6
3,L00000049012333,AS046594,2018-04-01,201804,6
4,L00000049012334,AS073285,2018-04-01,201804,6
...,...,...,...,...,...
197423,L00000049209753,TS977703,2019-03-31,201903,6
197424,L00000049209754,TS979550,2019-03-31,201903,6
197425,L00000049209755,TS995299,2019-03-31,201903,6
197426,L00000049209756,TS995853,2019-03-31,201903,6


In [61]:
uselog_weekday = uselog.groupby(['customer_id','년월','weekday'],
                                as_index = False).count()[['customer_id','년월','weekday','log_id']]

In [62]:
uselog_weekday.rename(columns={'log_id':'count'}, inplace = True )
uselog_weekday

,customer_id,년월,weekday,count
0,AS002855,201804,5,4
1,AS002855,201805,2,1
2,AS002855,201805,5,4
3,AS002855,201806,5,5
4,AS002855,201807,1,1
...,...,...,...,...
93328,TS999855,201901,1,1
93329,TS999855,201901,5,4
93330,TS999855,201901,6,1
93331,TS999855,201902,5,4


In [63]:
uselog_weekday = uselog_weekday.groupby('customer_id', as_index = False).max()[['customer_id', 'count']]
uselog_weekday

,customer_id,count
0,AS002855,5
1,AS008805,4
2,AS009013,2
3,AS009373,5
4,AS015233,5
...,...,...
4187,TS995853,5
4188,TS998593,5
4189,TS999079,5
4190,TS999231,5


In [64]:
uselog_weekday['routine_flg'] = 0

In [65]:
uselog_weekday["routine_flg"] = uselog_weekday["routine_flg"].\
                                            where(uselog_weekday["count"]<4, 1)
uselog_weekday.head()

,customer_id,count,routine_flg
0,AS002855,5,1
1,AS008805,4,1
2,AS009013,2,0
3,AS009373,5,1
4,AS015233,5,1
